In [0]:
data = [("ram","100","5000","male"), ("ravi","200","6000","male"),("raghu","300","4000","male")]
columns = ["name", "id", "salary","gender"]

df = spark.createDataFrame(data = data, schema = columns)
display(df)

name,id,salary,gender
ram,100,5000,male
ravi,200,6000,male
raghu,300,4000,male


In [0]:
df.write.saveAsTable("emp_info")

In [0]:
%sql
select * from emp_info

name,id,salary,gender
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


In [0]:
display(spark.sql("select * from emp_info"))

name,id,salary,gender
ramya,400,7000,female
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


In [0]:
df1 = spark.table("emp_info")
display(df1)

name,id,salary,gender
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


In [0]:
data = [("ramya","400","7000","female")]
columns = ["name", "id", "salary","gender"]
df2 = spark.createDataFrame(data = data, schema = columns)
df2.display()


name,id,salary,gender
ramya,400,7000,female


In [0]:
df2.write.insertInto("emp_info", overwrite=False)
display(df2)

name,id,salary,gender
ramya,400,7000,female


In [0]:
df1.write.insertInto("emp_info", overwrite=True)
display(spark.sql("select * from emp_info"))

name,id,salary,gender
ramya,400,7000,female
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


In [0]:
display(df1)
display(df2)

name,id,salary,gender
ramya,400,7000,female
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


name,id,salary,gender
ramya,400,7000,female


In [0]:
df.write.insertInto("emp_info", overwrite=True)
display(spark.sql("select * from emp_info"))

name,id,salary,gender
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


In [0]:
df2.createOrReplaceTempView("temp_view")

In [0]:
%sql
select * from temp_view

name,id,salary,gender
ramya,400,7000,female


In [0]:
%sql
insert into default.emp_info
select * from temp_view

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from emp_info

name,id,salary,gender
ramya,400,7000,female
ramya,400,7000,female
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


In [0]:
from pyspark.sql.functions import row_number,col,rank
from pyspark.sql.window import Window
y = Window.partitionBy("gender").orderBy(col("salary").desc())
x = df1.withColumn("rank", rank().over(y))
display(x)



name,id,salary,gender,rank
ramya,400,7000,female,1
ramya,400,7000,female,1
ravi,200,6000,male,1
ram,100,5000,male,2
raghu,300,4000,male,3


In [0]:
x.write.saveAsTable("x_table") 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2959641066037249>:1
----> 1 x.write.saveAsTable("x_table")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1518 if format is not None:
   1519     self.format(format)
-> 1520 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *args)
   1315 command = proto.CALL_COMMAND

In [0]:
%sql
select * from x_table

name,id,salary,gender,rank
ramya,400,7000,female,1
ramya,400,7000,female,1
ravi,200,6000,male,1
ram,100,5000,male,2
raghu,300,4000,male,3


In [0]:
%sql
select gender,max(salary) as max_salary from x_table
group by(gender)

gender,max_salary
female,7000
male,6000


In [0]:
from pyspark.sql.functions import lag
df1.withColumn("lag_values", lag("salary",2).over(y))\
    .display()

name,id,salary,gender,lag_values
ramya,400,7000,female,null
ramya,400,7000,female,null
ravi,200,6000,male,null
ram,100,5000,male,null
raghu,300,4000,male,6000


In [0]:
df1.display()

name,id,salary,gender
ramya,400,7000,female
ramya,400,7000,female
raghu,300,4000,male
ravi,200,6000,male
ram,100,5000,male


In [0]:
from pyspark.sql.functions import lead
(df.withColumn("lead_values", lead("salary").over(y))).display()


name,id,salary,gender,lead_values
ravi,200,6000,male,5000
ram,100,5000,male,4000
raghu,300,4000,male,null
